# Intro to asyncio - croutines

## Subgenerators with yield from

The `yield from` expression syntax was introduced in Python 3.3 to allow a generator to delegate work to a subgenerator.

Before `yield from` was introduced, we used a for loop when a generator needed to yield values produced from another generator:

In [ ]:
def sub_gen():
    yield 1.1
    yield 1.2

In [ ]:
def gen():
    yield 1
    for i in sub_gen():
        yield i
    yield 2

In [ ]:
for x in gen():
    print(x)

We can get the same result using `yield from`:

In [ ]:
def sub_gen():
    yield 1.1
    yield 1.2

def gen():
    yield 1
    yield from sub_gen()
    yield 2
    
for x in gen():
    print(x)

In Example, the for loop is the client code, gen is the delegating generator, and sub_gen is the subgenerator. Note that yield from **pauses gen**, and sub_gen takes over until it is exhausted. The **values yielded by `sub_gen` pass through gen directly to the client for loop**. Meanwhile, **gen is suspended and cannot see the values passing through it**. Only when sub_gen is done, gen resumes.

When the subgenerator contains a `return` statement with a value, that **value can be captured in the delegating generator** by using yield from as part of an expression. Example 17-26 demonstrates.

In [ ]:
def sub_gen():
    yield 1.1
    yield 1.2
    return 'Done!'

def gen():
    yield 1
    result = yield from sub_gen()
    print('<--', result)
    yield 2
    
for x in gen():
    print(x)

Now that we’ve seen the basics of yield from, let’s study a couple of simple but practical examples of its use.

### Reinventing chain

We saw in Table 17-3 that `itertools` provides a `chain` generator that yields items from several iterables, iterating over the first, then the second, and so on up to the last. This is a homemade implementation of chain with nested for loops in Python:

In [ ]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i
            
s = 'ABC'
r = range(3)
list(chain(s, r))

The chain generator in the preceding code is delegating to each iterable it in turn, by driving each it in the inner for loop. That inner loop can be replaced with a yield from expression, as shown in the next console listing:

In [ ]:
def chain(*iterables):
    for i in iterables:
        yield from i

In [ ]:
list(chain(s, r))

The use of yield from in this example is correct, and the code reads better, but it seems like syntactic sugar with little real gain. Now let’s develop a more interesting example.

### Traversing a Tree

In this section, we’ll see yield from in a script to traverse a tree structure. I will build it in baby steps.

The tree structure for this example is Python’s exception hierarchy. But the pattern can be adapted to show a directory tree or any other tree structure.

Starting from BaseException at level zero, the exception hierarchy is five levels deep as of Python 3.10. Our first baby step is to show level zero.

Given a root class, the tree generator in Example 17-27 yields its name and stops.

In [ ]:
def tree(cls):
    yield cls.__name__


def display(cls):
    for cls_name in tree(cls):
        print(cls_name)

if __name__ == '__main__':
    display(BaseException)

The next baby step takes us to level 1. The tree generator will yield the name of the root class and the names of each direct subclass. The names of the subclasses are indented to reveal the hierarchy. This is the output we want:

In [ ]:
def tree(cls):
    yield cls.__name__, 0                  
    for sub_cls in cls.__subclasses__():         
        yield sub_cls.__name__, 1                


def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level                 
        print(f'{indent}{cls_name}')


if __name__ == '__main__':
    display(BaseException)

- To support the indented output, yield the name of the class and its level in the hierarchy.
- Use the `__subclasses__` special method to get a list of subclasses.
- Yield name of subclass and level 1.
- Build indentation string of 4 spaces times level. At level zero, this will be an empty string.

In Example 17-29, I refactor tree to separate the special case of the root class from the subclasses, which are now handled in the sub_tree generator. At yield from, the tree generator is suspended, and sub_tree takes over yielding values.

In [ ]:
def tree(cls):
    yield cls.__name__, 0
    yield from sub_tree(cls)              


def sub_tree(cls):
    for sub_cls in cls.__subclasses__():
        yield sub_cls.__name__, 1         


def display(cls):
    for cls_name, level in tree(cls):     
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')


if __name__ == '__main__':
    display(BaseException)

- Delegate to sub_tree to yield the names of the subclasses.
- Yield the name of each subclass and level 1. Because of the yield from sub_tree(cls) inside tree, these values bypass the tree generator function completely… … and are received directly here.

In keeping with the baby steps method, I’ll write the simplest code I can imagine to reach level 2. For depth-first tree traversal, after yielding each node in level 1, I want to yield the children of that node in level 2, before resuming level 1. A nested for loop takes care of that, as in Example 17-30.

In [ ]:
def tree(cls):
    yield cls.__name__, 0
    yield from sub_tree(cls)


def sub_tree(cls):
    for sub_cls in cls.__subclasses__():
        yield sub_cls.__name__, 1
        for sub_sub_cls in sub_cls.__subclasses__():
            yield sub_sub_cls.__name__, 2


def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')


if __name__ == '__main__':
    display(BaseException)

You may already know where this is going, but I will stick to baby steps one more time: let’s reach level 3 by adding yet another nested for loop. The rest of the program is unchanged, so Example 17-31 shows only the sub_tree generator.

In [ ]:
def tree(cls):
    yield cls.__name__, 0
    yield from sub_tree(cls)


def sub_tree(cls):
    for sub_cls in cls.__subclasses__():
        yield sub_cls.__name__, 1
        for sub_sub_cls in sub_cls.__subclasses__():
            yield sub_sub_cls.__name__, 2
            for sub_sub_sub_cls in sub_sub_cls.__subclasses__():
                yield sub_sub_sub_cls.__name__, 3


def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')


if __name__ == '__main__':
    display(BaseException)

There is a clear pattern in Example 17-31. We do a for loop to get the subclasses of level N. Each time around the loop, we yield a subclass of level N, then start another for loop to visit level N+1.

In “Reinventing chain”, we saw how we can replace a nested for loop driving a generator with yield from on the same generator. We can apply that idea here, if we make sub_tree accept a level parameter, and yield from it recursively, passing the current subclass as the new root class with the next level number. See Example 17-32.

In [ ]:
def tree(cls):
    yield cls.__name__, 0
    yield from sub_tree(cls, 1)


def sub_tree(cls, level):
    for sub_cls in cls.__subclasses__():
        yield sub_cls.__name__, level
        yield from sub_tree(sub_cls, level+1)


def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')


if __name__ == '__main__':
    display(BaseException)

Example 17-32 can traverse trees of any depth, limited only by Python’s recursion limit. The default limit allows 1,000 pending functions.

Any good tutorial about recursion will stress the importance of having a base case to avoid infinite recursion. A base case is a conditional branch that returns without making a recursive call. The base case is often implemented with an if statement. In Example 17-32, sub_tree has no if, but there is an implicit conditional in the for loop: `if cls.__subclasses__()` returns an empty list, the body of the loop is not executed, therefore no recursive call happens. The base case is when the cls class has no subclasses. In that case, sub_tree yields nothing. It just returns.

Example 17-32 works as intended, but we can make it more concise by recalling the pattern we observed when we reached level 3 (Example 17-31): we yield a subclass with level N, then start a nested for loop to visit level N+1. In Example 17-32 we replaced that nested loop with yield from. Now we can merge tree and sub_tree into a single generator. Example 17-33 is the last step for this example.


In [ ]:
def tree(cls, level=0):
    yield cls.__name__, level
    for sub_cls in cls.__subclasses__():
        yield from tree(sub_cls, level+1)


def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')


if __name__ == '__main__':
    display(BaseException)

At the start of “Subgenerators with yield from”, we saw how yield from connects the subgenerator directly to the client code, bypassing the delegating generator. That connection becomes really important when generators are used as coroutines and not only produce but also consume values from the client code, as we’ll see in “Classic Coroutines”.

## Classic Coroutines

Understanding classic coroutines in Python is confusing because they are **actually generators used in a different way**. So let’s step back and consider another feature of Python that can be used in two ways.



Generators are commonly used as iterators, but they can also be used as coroutines. A **coroutine is really a generator function, created with the yield keyword in its body**. And a coroutine object is physically a generator object. Despite sharing the same underlying implementation in C, the use cases of generators and coroutines in Python are different.

David Beazley created some of the best talks and most comprehensive workshops about classic coroutines. In his PyCon 2009 course handout, he has a slide titled “Keeping It Straight,” which reads:
- Generators produce data for iteration
- Coroutines are consumers of data
- To keep your brain from exploding, don’t mix the two concepts together
- Coroutines are not related to iteration
- Note: There is a use of having `yield` produce a value in a coroutine, but it’s not tied to iteration.

Now let’s see how classic coroutines work.

### Basic Behavior of a Generator Used as a Coroutine

Simplest possible demonstration of coroutine in action:

In [18]:
def simple_coroutine():  # (1)
    print('-> coroutine started')
    x = yield  # (2)
    print('-> coroutine received:', x)

In [19]:
my_coro = simple_coroutine()

In [20]:
my_coro

<generator object simple_coroutine at 0x7f4985114c10>

In [21]:
next(my_coro)

-> coroutine started


In [22]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

1. A coroutine is defined as a generator function: with yield in its body.
2. yield is used in an expression; when the coroutine is designed just to receive data from the client it yields None—this is implicit because there is no expression to the right of the yield keyword.
3. As usual with generators, you call the function to get a generator object back.
4. The first call is next(…) because the generator hasn’t started so it’s not waiting in a yield and we can’t send it any data initially.
5. This call makes the yield in the coroutine body evaluate to 42; now the coroutine resumes and runs until the next yield or termination.
6. In this case, control flows off the end of the coroutine body, which prompts the generator machinery to raise StopIteration, as usual.

A coroutine can be in one of four states. You can determine the current state using the `inspect.getgeneratorstate(…)` function, which returns one of these strings:
- `GEN_CREATED` - Waiting to start execution.
- `GEN_RUNNING` - Currently being executed by the interpreter
- `GEN_SUSPENDED` - Currently suspended at a yield expression.
- `GEN_CLOSED` - Execution has completed.

Because the argument to the send method will become the value of the pending yield expression, it follows that you can only make a call like my_coro.send(42) if the coroutine is currently suspended. But that’s not the case if the coroutine has never been activated when its state is 'GEN_CREATED'. That’s why the first activation of a coroutine is always done with next(my_coro)—you can also call my_coro.send(None), and the effect is the same.

If you create a coroutine object and immediately try to send it a value that is not None, this is what happens:

In [23]:
my_coro = simple_coroutine()

In [24]:
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

Note the error message: it’s quite clear.

The initial call next(my_coro) is often described as "priming" the coroutine (i.e., advancing it to the first yield to make it ready for use as a live coroutine).

To get a better feel for the behavior of a coroutine, an example that yields more than once is useful.

In [25]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [26]:
my_coro2 = simple_coro2(14)

In [27]:
from inspect import getgeneratorstate

In [28]:
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [29]:
next(my_coro2)

-> Started: a = 14


14

In [30]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [31]:
my_coro2.send(28)

-> Received: b = 28


42

In [32]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [33]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

It’s crucial to understand that the execution of the coroutine is suspended exactly at the yield keyword. As mentioned before, in an **assignment statement, the code to the right of the = is evaluated before the actual assignment happens**. This means that in a line like b = yield a, the value of b will only be set when the coroutine is activated later by the client code. It takes some effort to get used to this fact, but understanding it is essential to make sense of the use of yield in asynchronous programming, as we’ll see later.

### Example: Coroutine to Compute a Running Average

While discussing closures in Chapter 9, we studied objects to compute a running average. Example 9-7 shows a class and Example 9-13 presents a higher-order function returning a function that keeps the total and count variables across invocations in a closure. Example 17-37 shows how to do the same with a coroutine.

In [3]:
from collections.abc import Generator

def averager() -> Generator[float, float, None]:  
    total = 0.0
    count = 0
    average = 0.0
    while True:  
        term = yield average  
        total += term
        count += 1
        average = total/count

- This function returns a generator that yields float values, accepts float values via .send(), and does not return a useful value.
- This infinite loop means the coroutine will keep on yielding averages as long as the client code sends values.
- The yield statement here suspends the coroutine, yields a result to the client, and—later—gets a value sent by the caller to the coroutine, starting another iteration of the infinite loop.

In a coroutine, total and count can be local variables: no instance attributes or closures are needed to keep the context while the coroutine is suspended waiting for the next .send(). That’s why coroutines are attractive replacements for callbacks in asynchronous programming—they keep local state between activations.

Example 17-38 runs doctests to show the averager coroutine in operation.

In [22]:
coro_avg = averager()

In [24]:
next(coro_avg)

0.0

In [25]:
coro_avg.send(10)

10.0

In [26]:
coro_avg.send(30)

20.0

In [27]:
coro_avg.send(5)

15.0

Create the coroutine object.

Start the coroutine. This yields the initial value of average: 0.0.

Now we are in business: each call to `.send()` yields the current average.

In Example 17-38, the call next(coro_avg) makes the coroutine advance to the yield, yielding the initial value for average. You can also start the coroutine by `calling coro_avg.send(None)` — this is actually what the next() built-in does. But you can’t send any value other than None, because the coroutine can only accept a sent value when it is suspended at a yield line. Calling `next()` or `.send(None)` to advance to the first yield is known as “**priming the coroutine**.”

After each activation, the coroutine is suspended precisely at the yield keyword, waiting for a value to be sent. The line `coro_avg.send(10)` provides that value, causing the coroutine to activate. The yield expression resolves to the value 10, assigning it to the term variable. The rest of the loop updates the total, count, and average variables. The next iteration in the while loop yields the average, and the coroutine is again suspended at the yield keyword.

The attentive reader may be anxious to know how the execution of an averager instance (e.g., coro_avg) may be terminated, because its body is an infinite loop. We don’t usually need to terminate a generator, because it is garbage collected as soon as there are no more valid references to it. If you need to explicitly terminate it, use the .close() method, as shown in Example 17-39.

In [28]:
coro_avg.send(20)

16.25

In [29]:
coro_avg.close() 

In [30]:
coro_avg.close() 

In [31]:
coro_avg.send(5) 

StopIteration: 

- coro_avg is the instance created in Example 17-38.
- The `.close()` method raises GeneratorExit at the suspended yield expression. If not handled in the coroutine function, the exception terminates it. GeneratorExit is caught by the generator object that wraps the coroutine—that’s why we don’t see it.
- Calling `.close()` on a previously closed coroutine has no effect.
- Trying `.send()` on a closed coroutine raises StopIteration.

### Returning a Value from a Coroutine

We’ll now study another coroutine to compute an average. This version will not yield partial results. Instead, it returns a tuple with the number of terms and the average. I’ve split the listing in two parts:

> Sentinel values are a special type of value. This value allows users to know when they are sending input. So this is a value that won’t be the part of the input to be processed. It is a value that is also useful to terminate the loop.

In [5]:
from collections.abc import Generator
from typing import Union, NamedTuple

class Result(NamedTuple):  
    count: int  # type: ignore  
    average: float

class Sentinel:  
    def __repr__(self):
        return f'<Sentinel>'

STOP = Sentinel()  

SendType = Union[float, Sentinel]

- the averager2 coroutine in Example 17-41 will return an instance of Result.
- The Result is actually a subclass of tuple, which has a .count() method that I don’t need. The # type: ignore comment prevents Mypy from complaining about having a count field
- A class to make a sentinel value with a readable `__repr__`.
- The sentinel value that I’ll use to make the coroutine stop collecting data and return a result.
- I’ll use this type alias for the second type parameter of the coroutine Generator return type, the SendType parameter.

Now, let’s study the coroutine code itself:

In [6]:
def averager2(verbose: bool = False) -> Generator[None, SendType, Result]:  
    total = 0.0
    count = 0
    average = 0.0
    while True:
        term = yield  
        if verbose:
            print('received:', term)
        if isinstance(term, Sentinel):  
            break
        total += term  
        count += 1
        average = total / count
    return Result(count, average)

- For this coroutine, the yield type is None because it does not yield data. It receives data of the SendType and returns a Result tuple when done.
- Using yield like this only makes sense in coroutines, which are designed to consume data. This yields None, but receives a term from .send(term).
- If the term is a Sentinel, break from the loop. Thanks to this isinstance check…
- …Mypy allows me to add term to the total without flagging an error that I can’t add a float to an object that may be a float or a Sentinel.
- This line will be reached only if a Sentinel is sent to the coroutine.



Now let’s see how we can use this coroutine, starting with a simple example that doesn’t actually produce a result:

In [7]:
coro_avg = averager2()

In [8]:
next(coro_avg)

In [9]:
coro_avg.send(10)  

In [10]:
coro_avg.send(30)

In [11]:
coro_avg.send(6.5)

In [12]:
coro_avg.close()

- Recall that averager2 does not yield partial results. It yields None, which Python’s console omits.
- Calling .close() in this coroutine makes it stop but does not return a result, because the GeneratorExit exception is raised at the yield line in the coroutine, so the return statement is never reached

Now let’s make it work:

In [14]:
coro_avg = averager2()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

In [ ]:
try:
    coro_avg.send(STOP)  
except StopIteration as exc:
    result = exc.value  
result 

- Sending the STOP sentinel makes the coroutine break from the loop and return a Result. The generator object that wraps the coroutine then raises StopIteration.
- The StopIteration instance has a value attribute bound to the value of the return statement that terminated the coroutine.


This idea of “smuggling” the return value out of the coroutine wrapped in a StopIteration exception is a bizarre hack. Nevertheless, this bizarre hack is part of PEP 342—Coroutines via Enhanced Generators, and is documented with the StopIteration exception, and in the “Yield expressions” section of Chapter 6 of The Python Language Reference.

A delegating generator can get the return value of a coroutine directly using the yield from syntax, as shown:

In [15]:
def compute():
    res = yield from averager2(True)  
    print('computed:', res)  
    return res  

comp = compute()

In [16]:
for v in [None, 10, 20, 30, STOP]:  
    try:
        comp.send(v)  
    except StopIteration as exc:  
        result = exc.value

received: 10
received: 20
received: 30
received: <Sentinel>
computed: Result(count=3, average=20.0)


In [17]:
result  

Result(count=3, average=20.0)

- res will collect the return value of averager2; the yield from machinery retrieves the return value when it handles the StopIteration exception that marks the termination of the coroutine. When True, the verbose parameter makes the coroutine print the value received, to make its operation visible.
- Keep an eye out for the output of this line when this generator runs.
- Return the result. This will also be wrapped in StopIteration.
- Create the delegating coroutine object.
- This loop will drive the delegating coroutine.
- First value sent is None, to prime the coroutine; last is the sentinel to stop it.
- Catch StopIteration to fetch the return value of compute.
- mAfter the lines output by averager2 and compute, we get the Result instance.

Even though the examples here don’t do much, the code is hard to follow. Driving the coroutine with .send() calls and retrieving results is complicated, except with yield from—but we can only use that syntax inside a delegating generator/coroutine, which must ultimately be driven by some nontrivial code, as shown in Example 17-44.

The previous examples show that using coroutines directly is cumbersome and confusing. Add exception handling and the coroutine .throw() method, and examples become even more convoluted. I won’t cover .throw() in this book because—like .send()—it is only useful to drive coroutines “by hand,” but I don’t recommend doing that, unless you are creating a new coroutine-based framework from scratch.

In practice, productive work with coroutines requires the support of a specialized framework. That is what asyncio provided for classic coroutines way back in Python 3.3. With the advent of native coroutines in Python 3.5, the Python core developers are gradually phasing out support for classic coroutines in asyncio. But the underlying mechanisms are very similar. The async def syntax makes native coroutines easier to spot in code, which is a great benefit. Inside, native coroutines use await instead of yield from to delegate to other coroutines. 